In [1]:
try:
    import chess
    import chess.engine
    import chess.pgn
    import time
    import random
    import uuid 
    import pandas as pd
    from stockfish import Stockfish
    from IPython.display import display, HTML, clear_output
    from html import escape
    from datetime import datetime
except:
    !pip install python-chess
    !pip install stockfish

In [2]:
engine = chess.engine.SimpleEngine.popen_uci("engine/stockfish_linux")
engine.configure({"Threads": 40})

In [3]:
def who(player):
    return "White" if player == chess.WHITE else "Black"

In [4]:
def display_board(board, use_svg):
    if use_svg:
        return board._repr_svg_()
    else:
        return "<pre>" + str(board) + "</pre>"

In [5]:
def staticAnalysis(board,color):
    score = 0
    my_color = str(color)
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 5), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 12), 
                           (chess.KNIGHT, 4),
                           (chess.ROOK, 6)]:
        if my_color =='White':
            score += len(board.pieces(piece, chess.WHITE)) * value
        else:
            score += len(board.pieces(piece, not chess.WHITE)) * value
    return score

In [6]:
def play_game(player1, player2, visual="svg", pause=0):
    use_svg = (visual == "svg")
    board = chess.Board()
    gameID = uuid.uuid1()
    gameTime = datetime.now().strftime("%d-%m-%Y %I:%M:%S %p")
    gameData = dict()
    gameDataList = list()
   
    try:
        while not board.is_game_over(claim_draw=True):
            if board.turn == chess.WHITE:
                uci = player1(board)
            else:
                uci = player2(board)
            oldBoard = board.fen()
            name = who(board.turn)
            board.push_uci(uci)
            board_stop = display_board(board, use_svg)
            
            scores_w = staticAnalysis(board,'White')
            scores_b = staticAnalysis(board,'Black')
                
            moveList = board.move_stack
            LenMove= len(moveList)
            temPGame = LenMove,{'old Board': oldBoard,'move': uci,'Who': name,'new board':board.fen(),'score white':scores_w,'score black':scores_b}
            gameDataList.append(temPGame)
            #game = chess.pgn.Game()
            html = "<h2>Chess With Deep Neural Network</h2><h3>%s is thinking...<br/><b>Move %s White : %s  Black : %s <br/> %s  Play '%s'</b><br/><h5/><br/>%s<br/>" % (who(board.turn), len(moveList),scores_w,scores_b, name, uci, board_stop)
            if visual is not None:
                if visual == "svg":
                    clear_output(wait=True)
                display(HTML(html))
                if visual == "svg":
                    time.sleep(pause)
    except KeyboardInterrupt:
        msg = "Game interrupted!"
        errors ="interrupted"
        return (None, msg, board)
    
    errors = None
    
    if board.is_checkmate():
        msg = "checkmate: " + who(not board.turn) + " wins!"
        win = who(not board.turn)

    elif board.is_stalemate():
        msg = "draw: stalemate"
        win = "stalemate"
    elif board.is_fivefold_repetition():
        msg = "draw: 5-fold repetition"
        win = "5-fold"
    elif board.is_insufficient_material():
        msg = "draw: insufficient material"
        win = "insufficient"
    elif board.can_claim_draw():
        msg = "draw: claim"
        win = "claim"
        
    if visual is not None:
        msgD = "<h2>%s<h2/>" % (msg)
        display(HTML(msgD))
        time.sleep(pause*10)
    
    gameData.update({
        'GameID':gameID,
        'GameTime':gameTime,
        'Game Results' : win,
        'Error':errors,
        'Game Messege' : msg,
        'Winning Board': board,
        'GameData': gameDataList
            })
    return gameData

In [7]:
def engine_move(board):
    result = engine.play(board, chess.engine.Limit(depth=25))
    results = result.move
    return results.uci()

In [8]:
play_game(engine_move , engine_move)

{'GameID': UUID('e522bf02-9df7-11ea-99fe-57a0b8bff344'),
 'GameTime': '24-05-2020 09:51:01 PM',
 'Game Results': 'claim',
 'Error': None,
 'Game Messege': 'draw: claim',
 'Winning Board': Board('3b1k2/8/R3p3/pB3pnp/P5p1/6P1/6KP/8 w - - 16 91'),
 'GameData': [(1,
   {'old Board': 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1',
    'move': 'd2d4',
    'Who': 'White',
    'new board': 'rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR b KQkq - 0 1',
    'score white': 50,
    'score black': 50}),
  (2,
   {'old Board': 'rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR b KQkq - 0 1',
    'move': 'g8f6',
    'Who': 'Black',
    'new board': 'rnbqkb1r/pppppppp/5n2/8/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 1 2',
    'score white': 50,
    'score black': 50}),
  (3,
   {'old Board': 'rnbqkb1r/pppppppp/5n2/8/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 1 2',
    'move': 'c2c4',
    'Who': 'White',
    'new board': 'rnbqkb1r/pppppppp/5n2/8/2PP4/8/PP2PPPP/RNBQKBNR b KQkq - 0 2',
    'score white': 50,
    'sco